In [1]:
# Instead of openai-agents, you would use libraries like these:
!pip install langchain-google-genai langchain-core langchain
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=78912c5bcc1ef721b705309a351d4ba9d2b68696f613547ac3652d6f10093b53
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


### Checks

In [2]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

## Guardrail & Triage Routing

In [1]:
# !pip install langchain langchain-google-genai wikipedia pydantic

import os
import asyncio
import textwrap
from typing import Literal

# Use pydantic's standard BaseModel
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
import wikipedia
from google.colab import userdata

# --- SETUP: DUMMY HR POLICY FILE ---
# This creates a fake HR policy file for the HR Agent to use.
hr_policy_content = """
# Company Leave Policy

## General Provisions
Full-time staff are entitled to 24 days of annual leave per year.
This accrues at a rate of 2 days per month.
Up to 5 unused days may be carried forward into the next year with manager approval.

## Sick Leave
Sick leave is determined by service duration.
- Up to 1 year of service: 2 weeks full pay, 2 weeks half pay.
- 1-3 years of service: 4 weeks full pay, 4 weeks half pay.
- Over 3 years of service: 8 weeks full pay, 8 weeks half pay.
A doctor's note is required for absences longer than 3 consecutive days.

## Special Leave
- Compassionate Leave: Up to 5 days per year for the loss of a close family member.
- Public Holidays: The company observes all official public holidays in the country of employment.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)
# ------------------------------------


# 2. Configure API Key and Initialize the LLM
# -------------------------------------------
llm = None  # Initialize llm to None
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    llm.invoke("Test query to validate API key")
    print("✅ Gemini API Key configured successfully.")

except Exception as e:
    print(f"❌ Error configuring Gemini API. Please ensure your key is correct. Details: {e}")
    # Exit if the LLM can't be configured
    exit()


# 3. Define Tools for the Agents
# ------------------------------
@tool
def wiki_tool(query: str) -> str:
    """
    Fetches a summary of the given query from Wikipedia.
    Use this for technical questions about concepts, people, or places.
    """
    try:
        summary = wikipedia.summary(query, auto_suggest=False, sentences=5)
        return summary
    except wikipedia.exceptions.PageError:
        return f"Could not find a Wikipedia page for '{query}'. Please try a different query."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"'{query}' is ambiguous. It could refer to: {e.options[:5]}. Please be more specific."
    except Exception as e:
        return f"An error occurred while fetching from Wikipedia: {str(e)}"

@tool
def file_search_tool(query: str) -> str:
    """
    Searches the content of the company's HR policy document.
    Use this for any questions about leave, benefits, or other HR-related policies.
    """
    print(f"\n🔎 Searching HR policy for: '{query}'...")
    with open("hr_policy.txt", "r") as f:
        policy_content = f.read()
    return policy_content


# 4. Define All Agents
# --------------------

# Agent 4a: Guardrail Agent (for input validation)
class GuardrailOutput(BaseModel):
    """
    Schema for the guardrail's decision.
    `is_valid` is true if the query is related to HR or technical topics.
    """
    is_valid: bool = Field(description="True if the user's query is about HR or a technical topic, False otherwise.")
    reasoning: str = Field(description="A brief, one-sentence explanation for the decision.")

guardrail_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a gatekeeper. Your job is to determine if a user's query is on-topic.
    The allowed topics are:
    1.  Human Resources (HR) and workplace policies (e.g., leave, benefits).
    2.  Technical subjects (e.g., AI, programming, science, software).

    Any other topic, such as entertainment, sports, or general chit-chat, is considered off-topic.
    You must respond only with the structured output."""),
    ("human", "User Query: {query}")
])

guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)


# Agent 4b: H.R. Agent
hr_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful H.R. assistant. You must use the `file_search_tool` to find information in the company policy document to answer the user's question. Answer based ONLY on the information retrieved from the tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
hr_tools = [file_search_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)


# Agent 4c: Technical Agent
tech_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful technical assistant. You must use the `wiki_tool` to answer the user's question. Provide a clear and concise explanation based on the information from the tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
tech_tools = [wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)


# 5. Define the Triage Agent (The Router)
# ---------------------------------------
class TriageDecision(BaseModel):
    """The decision on which specialist agent to route the user's query to."""
    agent: Literal["HR", "Technical"] = Field(description="The name of the agent to route to, either 'HR' or 'Technical'.")

triage_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a triage agent. Your job is to analyze the user's query and decide which specialist is best suited to handle it. Do not answer the question yourself.

    - If the query is about company policies, benefits, leave, or other workplace matters, route to 'HR'.
    - If the query is about a concept, technology, or any general knowledge topic, route to 'Technical'.

    You must respond only with the structured output."""),
    ("human", "User Query: {query}")
])

triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 6. Orchestrate the System
# -------------------------
async def run_agent_system(query: str):
    """
    Main function to run the multi-agent system.
    1. Checks if the query is valid using the Guardrail.
    2. If valid, uses the Triage agent to select the correct specialist.
    3. Invokes the selected specialist agent to get the final answer.
    """
    print("="*60)
    print(f"👤 User Query: {query}")
    print("="*60)

    # --- Step 1: Guardrail Check ---
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})

    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return

    print(f"✅ Guardrail Passed. Reason: {guardrail_result.reasoning}")

    # --- Step 2: Triage Routing ---
    print("\n🚦 Running Triage Agent to select specialist...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")

    # --- Step 3: Invoke Specialist Agent ---
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    elif selected_agent == "Technical":
        result = await tech_agent_executor.ainvoke({"input": query})
    else:
        result = {"output": "Error: Triage agent selected an unknown specialist."}

    final_answer = result.get('output', 'Sorry, I could not process your request.')

    print("\n" + "-"*60)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*60 + "\n")


# 7. Run Example Queries
# ----------------------
async def main():
    # Example 1: A technical question (should be routed to Technical Agent)
    await run_agent_system("What is Agentic AI?")

    # Example 2: An HR question (should be routed to HR Agent)
    await run_agent_system("How many sick days do I get after working here for two years?")

    # Example 3: An out-of-scope question (should be blocked by Guardrail)
    await run_agent_system("Suggest some good bollywood movies from the 1990s.")


if llm: # Only run if the LLM was successfully initialized
    await main()

✅ Gemini API Key configured successfully.
👤 User Query: What is Agentic AI?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: The query is about a technical topic (AI).

🚦 Running Triage Agent to select specialist...
🎯 Specialist selected: Technical

▶️  Invoking Technical Agent...

------------------------------------------------------------
🤖 Final Answer:
Agentic AI refers to artificial intelligence systems designed to operate
autonomously, making decisions and performing tasks without direct human
intervention.  These systems react independently to conditions to produce
results.  It's closely related to agentic automation (or agent-based process
management systems) when applied to process automation.  Applications span
various fields including software development, customer support, cybersecurity,
and business intelligence.
------------------------------------------------------------

👤 User Query: How many sick days do I get after working here for two years?
🛡️  Running G

## Integrating with RAG

In [2]:
!pip install langchain langchain-google-genai wikipedia pydantic
!pip install langchain-community faiss-cpu # Add these for RAG

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [3]:
import os
import asyncio
import textwrap
from typing import Literal
from google.colab import userdata

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --- SETUP: EXPANDED DUMMY HR POLICY FILE ---
# We'll add more distinct sections to make the RAG process more meaningful.
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration.
- Up to 1 year of service: 2 weeks full pay, 2 weeks half pay.
- 1-3 years of service: 4 weeks full pay, 4 weeks half pay.
- Over 3 years of service: 8 weeks full pay, 8 weeks half pay.
A doctor's note is required for absences longer than 3 consecutive days.

## Compassionate Leave
Special leave for bereavement is available. Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member (spouse, child, parent, sibling).

## Public Holidays
The company observes all official public holidays in the country of employment. These days are paid leave and do not count against an employee's annual leave balance.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week with manager approval. WFH arrangements are based on job role and performance. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)
# ------------------------------------


# 2. Configure API Key and Initialize the LLM
# -------------------------------------------
llm = None
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. NEW: Setup RAG Pipeline for HR Agent
# -----------------------------------------
retriever = None
try:
    print("\n⚙️  Setting up RAG pipeline for HR Agent...")
    # a. Load the documents
    loader = TextLoader("./hr_policy.txt")
    documents = loader.load()

    # b. Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)
    print(f"📄 Split policy into {len(docs)} chunks.")

    # c. Create embeddings and store in a FAISS vector store
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_documents(docs, embeddings)

    # d. Create a retriever from the vector store
    retriever = vector_store.as_retriever(search_kwargs={"k": 2}) # Retrieve top 2 most relevant chunks
    print("✅ RAG pipeline setup complete.")
except Exception as e:
    print(f"❌ Error setting up RAG pipeline: {e}")
    exit()


# 4. Define Tools for the Agents
# ------------------------------
@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary of the given query from Wikipedia."""
    try:
        summary = wikipedia.summary(query, auto_suggest=False, sentences=5)
        return summary
    except Exception as e:
        return f"An error occurred while fetching from Wikipedia: {str(e)}"

@tool
def hr_rag_tool(query: str) -> str:
    """
    Searches the HR knowledge base for policies on leave, benefits, and workplace rules.
    Use this to answer any HR-related questions.
    """
    print(f"\n🔎 Retrieving HR context for: '{query}'...")
    docs = retriever.invoke(query)
    # Format the retrieved documents into a single string
    context = "\n\n".join(doc.page_content for doc in docs)
    return f"Retrieved context:\n{context}"


# 5. Define All Agents (Guardrail, HR, Technical, Triage)
# --------------------------------------------------------
# Guardrail, Technical, and Triage agents remain unchanged.
# We only need to update the HR agent to use the new RAG tool.

# Agent 5a: Guardrail Agent
class GuardrailOutput(BaseModel):
    is_valid: bool = Field(description="True if query is about HR/tech, False otherwise.")
    reasoning: str = Field(description="A brief explanation for the decision.")
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "You are a gatekeeper. Your job is to determine if a user's query is on-topic (HR or Technical). Respond only with the structured output."), ("human", "User Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

# Agent 5b: H.R. Agent (Now powered by RAG)
hr_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful H.R. assistant. You must use the `hr_rag_tool` to find information in the company policy document. Answer the user's question based ONLY on the retrieved context from the tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
hr_tools = [hr_rag_tool] # Use the new RAG tool
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

# Agent 5c: Technical Agent
tech_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a technical assistant. Use the `wiki_tool` to answer the user's question."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
tech_tools = [wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)

# Agent 5d: Triage Agent
class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "You are a triage agent. Analyze the user's query and decide if it should be handled by 'HR' or 'Technical'. Do not answer the question. Respond only with the structured output."), ("human", "User Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 6. Orchestrate the System
# -------------------------
async def run_agent_system(query: str):
    print("="*60)
    print(f"👤 User Query: {query}")
    print("="*60)

    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed. Reason: {guardrail_result.reasoning}")

    print("\n🚦 Running Triage Agent to select specialist...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")

    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})

    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*60)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*60 + "\n")


# 7. Run Example Queries
# ----------------------
async def main():
    # Example 1: Technical question
    await run_agent_system("What is the difference between AI, Machine Learning, and Deep Learning?")

    # Example 2: HR question that requires specific retrieval via RAG
    await run_agent_system("What is the policy for working from home and do I get any money for setup?")

    # Example 3: Another HR question
    await run_agent_system("I need to take time off because my father passed away. What should I do?")

    # Example 4: Out-of-scope question
    await run_agent_system("Can you recommend a good place for lunch in Mumbai?")


if __name__ == "__main__":
    if llm and retriever: # Only run if LLM and RAG retriever were initialized
        # In a Jupyter/Colab notebook, await the main function directly
        # await main()
        # In a standard .py script, use asyncio.run()
        try:
             asyncio.run(main())
        except RuntimeError: # Handles the case of running in a notebook
             await main()

✅ Gemini API Key configured successfully.

⚙️  Setting up RAG pipeline for HR Agent...
📄 Split policy into 2 chunks.
✅ RAG pipeline setup complete.
👤 User Query: What is the difference between AI, Machine Learning, and Deep Learning?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: Query is about a technical topic (AI, ML, DL).

🚦 Running Triage Agent to select specialist...
🎯 Specialist selected: Technical

▶️  Invoking Technical Agent...

------------------------------------------------------------
🤖 Final Answer:
I am sorry, I could not find a Wikipedia page that directly addresses the
difference between AI, Machine Learning, and Deep Learning.  However, I can
explain the relationship between these concepts.  Artificial Intelligence (AI)
is the broadest concept, encompassing any technique that enables computers to
mimic human intelligence.  This includes a wide range of approaches, from simple
rule-based systems to complex algorithms.  Machine Learning (ML) is a subset of


/tmp/ipython-input-90055898.py:202: RuntimeWarning: coroutine 'main' was never awaited
  await main()


## Parent Document Retriever (Hierarchical RAG)

- InMemoryStore to hold the parent documents.
- child_splitter to create small, precise chunks for searching and a parent_splitter to create the larger, context-rich chunks that will be retrieved.

In [3]:
!pip install langchain langchain-google-genai wikipedia pydantic -q
!pip install langchain-community faiss-cpu -q

In [1]:
import os
import asyncio
import textwrap
from typing import Literal, List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --- SETUP: HR POLICY FILE ---
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration.
- Up to 1 year of service: 2 weeks full pay, 2 weeks half pay.
- 1-3 years of service: 4 weeks full pay, 4 weeks half pay.
- Over 3 years of service: 8 weeks full pay, 8 weeks half pay.
A doctor's note is required for absences longer than 3 consecutive days.

## Compassionate Leave
Special leave for bereavement is available. Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member (spouse, child, parent, sibling).

## Public Holidays
The company observes all official public holidays in the country of employment. These days are paid leave and do not count against an employee's annual leave balance.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week with manager approval. WFH arrangements are based on job role and performance. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. NEW: Robust Parent-Child RAG Setup
# ---------------------------------------
retriever = None
try:
    print("\n⚙️  Setting up Robust Parent-Child RAG pipeline...")
    # a. Load the documents
    loader = TextLoader("./hr_policy.txt")
    docs = loader.load()

    # b. Define the splitters
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

    # c. Split documents into parent chunks
    parent_documents = parent_splitter.split_documents(docs)

    # d. Create child documents from parents and store parent text in metadata
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            # Create a new Document for each child with parent's content in metadata
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    print(f"📄 Created {len(parent_documents)} parent chunks and {len(child_documents)} child chunks for indexing.")

    # e. Create a standard, reliable FAISS vector store from the child documents
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)

    # f. Create a standard retriever
    retriever = vectorstore.as_retriever()
    print("✅ Robust RAG pipeline setup complete.")

except Exception as e:
    print(f"❌ Error setting up RAG pipeline: {e}")
    exit()


# 4. Define Tools for the Agents
# ------------------------------
@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary of the given query from Wikipedia."""
    try:
        summary = wikipedia.summary(query, auto_suggest=False, sentences=5)
        return summary
    except Exception as e:
        return f"An error occurred while fetching from Wikipedia: {str(e)}"

@tool
def hr_rag_tool(query: str) -> str:
    """
    Searches the HR knowledge base for policies on leave, benefits, and workplace rules.
    Use this to answer any HR-related questions.
    """
    print(f"\n🔎 Retrieving HR context for: '{query}'...")
    # 1. Retrieve the small, specific child documents
    child_docs = retriever.invoke(query)

    # 2. Extract the parent content from the metadata of the retrieved child documents
    # Use a set to avoid returning duplicate parent documents
    parent_contents = {doc.metadata['parent_content'] for doc in child_docs}

    # 3. Format the unique parent documents as the context
    context = "\n\n".join(parent_contents)
    return f"Retrieved context:\n{context}"


# 5. Define All Agents (No changes here)
# ----------------------------------------
class GuardrailOutput(BaseModel):
    is_valid: bool = Field(description="True if query is about HR/tech, False otherwise.")
    reasoning: str = Field(description="A brief explanation for the decision.")
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "You are a gatekeeper. Your job is to determine if a user's query is on-topic (HR or Technical). Respond only with the structured output."), ("human", "User Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

hr_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful H.R. assistant. You must use the `hr_rag_tool` to find information in the company policy document. Answer the user's question based ONLY on the retrieved context from the tool."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

tech_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a technical assistant. Use the `wiki_tool` to answer the user's question."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
tech_tools = [wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)

class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "You are a triage agent. Analyze the user's query and decide if it should be handled by 'HR' or 'Technical'. Respond only with the structured output."), ("human", "User Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 6. Orchestrate the System (No changes here)
# ---------------------------------------------
async def run_agent_system(query: str):
    print("="*60)
    print(f"👤 User Query: {query}")
    print("="*60)
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed. Reason: {guardrail_result.reasoning}")
    print("\n🚦 Running Triage Agent to select specialist...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*60)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*60 + "\n")


# 7. Run Example Queries
# ----------------------
async def main():
    await run_agent_system("What is the difference between AI, Machine Learning, and Deep Learning?")
    await run_agent_system("I've been sick for 4 days. Do I need to provide any specific documentation?")
    await run_agent_system("Where can I find the best vada pav in Mumbai?")


if __name__ == "__main__":
    if llm and retriever:
        try:
             await main() # Use this in a notebook
        except NameError: # Fallback for .py script
             asyncio.run(main())

✅ Gemini API Key configured successfully.

⚙️  Setting up Robust Parent-Child RAG pipeline...
📄 Created 1 parent chunks and 4 child chunks for indexing.
✅ Robust RAG pipeline setup complete.
👤 User Query: What is the difference between AI, Machine Learning, and Deep Learning?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: Query is about a technical topic

🚦 Running Triage Agent to select specialist...
🎯 Specialist selected: Technical

▶️  Invoking Technical Agent...

------------------------------------------------------------
🤖 Final Answer:
I am sorry, I could not find a Wikipedia page with information on the difference
between AI, Machine Learning, and Deep Learning.  I need more information to
answer your question.
------------------------------------------------------------

👤 User Query: I've been sick for 4 days. Do I need to provide any specific documentation?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: Query is about HR policy.

🚦 Running Triage Agen

## Integrateing Query Expansion into our RAG pipeline.

In [2]:
import os
import asyncio
import textwrap
from typing import Literal, List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --- SETUP: HR POLICY FILE ---
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration.
- Up to 1 year of service: 2 weeks full pay, 2 weeks half pay.
- 1-3 years of service: 4 weeks full pay, 4 weeks half pay.
- Over 3 years of service: 8 weeks full pay, 8 weeks half pay.
A doctor's note is required for absences longer than 3 consecutive days.

## Compassionate Leave
Special leave for bereavement is available. Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member (spouse, child, parent, sibling).

## Public Holidays
The company observes all official public holidays in the country of employment. These days are paid leave and do not count against an employee's annual leave balance.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week with manager approval. WFH arrangements are based on job role and performance. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0) # Temp=0 for predictable output
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. Robust Parent-Child RAG Setup (No changes here)
# --------------------------------------------------
retriever = None
try:
    print("\n⚙️  Setting up Robust Parent-Child RAG pipeline...")
    loader = TextLoader("./hr_policy.txt")
    docs = loader.load()
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    parent_documents = parent_splitter.split_documents(docs)
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    print(f"📄 Created {len(parent_documents)} parent chunks and {len(child_documents)} child chunks for indexing.")
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)
    retriever = vectorstore.as_retriever()
    print("✅ Robust RAG pipeline setup complete.")
except Exception as e:
    print(f"❌ Error setting up RAG pipeline: {e}")
    exit()


# 4. Define Tools for the Agents
# ------------------------------

# --- NEW: Query Expansion Chain ---
# This chain will take a query and generate alternative versions of it.
query_expansion_prompt = ChatPromptTemplate.from_template(
    """You are an expert at query expansion. Your task is to rewrite a given user query into 3 alternative, more detailed versions.
The goal is to improve the recall of a vector search.
Provide the queries on new lines. DO NOT number the questions.

Original Query:
{query}

Your Expanded Queries:
"""
)

query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()
# ------------------------------------

@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary of the given query from Wikipedia."""
    try:
        summary = wikipedia.summary(query, auto_suggest=False, sentences=5)
        return summary
    except Exception as e:
        return f"An error occurred while fetching from Wikipedia: {str(e)}"

@tool
def hr_rag_tool(query: str) -> str:
    """
    Searches the HR knowledge base for policies on leave, benefits, and workplace rules.
    Use this to answer any HR-related questions.
    """
    print(f"\n🔎 Original HR query: '{query}'")
    # 1. Generate expanded queries
    expanded_queries_str = query_expansion_chain.invoke({"query": query})
    expanded_queries = expanded_queries_str.strip().split('\n')
    all_queries = [query] + expanded_queries
    print(f"🔍 Performing retrieval with expanded queries: {all_queries}")

    # 2. Retrieve documents for all queries
    all_retrieved_docs = []
    for q in all_queries:
        all_retrieved_docs.extend(retriever.invoke(q))

    # 3. Get the unique parent documents from the retrieved child documents
    unique_parent_contents = {doc.metadata['parent_content'] for doc in all_retrieved_docs}

    # 4. Format the unique parent documents as the context
    context = "\n\n".join(unique_parent_contents)
    return f"Retrieved context:\n{context}"


# 5. Define All Agents (No changes here)
# ----------------------------------------
class GuardrailOutput(BaseModel):
    is_valid: bool = Field(description="True if query is about HR/tech, False otherwise.")
    reasoning: str = Field(description="A brief explanation for the decision.")
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "You are a gatekeeper. Your job is to determine if a user's query is on-topic (HR or Technical). Respond only with the structured output."), ("human", "User Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

hr_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful H.R. assistant. You must use the `hr_rag_tool` to find information in the company policy document. Answer the user's question based ONLY on the retrieved context from the tool."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

tech_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a technical assistant. Use the `wiki_tool` to answer the user's question."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
tech_tools = [wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)

class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "You are a triage agent. Analyze the user's query and decide if it should be handled by 'HR' or 'Technical'. Respond only with the structured output."), ("human", "User Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 6. Orchestrate the System (No changes here)
# ---------------------------------------------
async def run_agent_system(query: str):
    print("="*60)
    print(f"👤 User Query: {query}")
    print("="*60)
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed. Reason: {guardrail_result.reasoning}")
    print("\n🚦 Running Triage Agent to select specialist...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*60)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*60 + "\n")


# 7. Run Example Queries
# ----------------------
async def main():
    await run_agent_system("What is the difference between AI, Machine Learning, and Deep Learning?")
    await run_agent_system("I've been sick for 4 days. What paperwork do I need?")
    await run_agent_system("Where can I find the best vada pav in Mumbai?")


if __name__ == "__main__":
    if llm and retriever:
        try:
             await main()
        except (NameError, RuntimeError):
             # This handles both cases: 'main' not defined outside of a notebook,
             # and 'asyncio.run' error inside a notebook.
             try:
                 asyncio.run(main())
             except RuntimeError:
                 # If we are in a notebook, the above will fail, so we just await it.
                 # This construct makes the script runnable in both .py and .ipynb
                 pass # The 'await main()' would have already been tried.

✅ Gemini API Key configured successfully.

⚙️  Setting up Robust Parent-Child RAG pipeline...
📄 Created 1 parent chunks and 4 child chunks for indexing.
✅ Robust RAG pipeline setup complete.
👤 User Query: What is the difference between AI, Machine Learning, and Deep Learning?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: Query is about a technical topic (AI, ML, DL).

🚦 Running Triage Agent to select specialist...
🎯 Specialist selected: Technical

▶️  Invoking Technical Agent...

------------------------------------------------------------
🤖 Final Answer:
I am sorry, I could not find a Wikipedia page with information on the
differences between AI, Machine Learning, and Deep Learning.  I need more
information to answer your question.
------------------------------------------------------------

👤 User Query: I've been sick for 4 days. What paperwork do I need?
🛡️  Running Guardrail Check...
✅ Guardrail Passed. Reason: Query is about HR.

🚦 Running Triage Agent to select spe

## Dual RAG Pipelines - Tech & HR

In [3]:
import os
import asyncio
import textwrap
from typing import Literal, List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --- SETUP: Create Dummy Knowledge Base Files ---

# HR Policy Document
hr_policy_content = """
# Company Leave Policy
...
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)

# NEW: Technical Knowledge Base Document
tech_docs_content = """
# Internal Technical Documentation

## Project Phoenix: Frontend
- Repository: git.corp.example.com/phoenix/frontend-app
- Language: TypeScript
- Framework: React
- Style Guide: Adhere to the 'StandardJS' style guide.
- Description: This is the main customer-facing web application.

## Project Phoenix: Backend
- Repository: git.corp.example.com/phoenix/backend-services
- Language: Python
- Framework: FastAPI
- Description: These are the microservices that power the Phoenix frontend.

## Deployment Procedures
- All deployments to production must go through the CI/CD pipeline in Jenkins.
- Request a production deployment by creating a JIRA ticket with the 'DEVOPS' component.
- Staging environment is open for all developers for testing. The URL is staging.phoenix.example.com.

## Coding Standards
- All Python code must be formatted with 'black'.
- All frontend code must pass 'eslint' checks before merging.
- API keys and secrets must never be hard-coded. They should be loaded from HashiCorp Vault.
"""
with open("tech_docs.txt", "w") as f:
    f.write(tech_docs_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. Setup RAG Pipelines
# ----------------------
hr_retriever = None
tech_retriever = None

# Function to create a RAG retriever to avoid code duplication
def create_rag_retriever(file_path: str, chunk_size: int, chunk_overlap: int) -> FAISS.as_retriever:
    loader = TextLoader(file_path)
    docs = loader.load()
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    parent_documents = parent_splitter.split_documents(docs)
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)
    return vectorstore.as_retriever()

try:
    # 3a. HR RAG Pipeline
    print("\n⚙️  Setting up HR RAG pipeline...")
    hr_retriever = create_rag_retriever("./hr_policy.txt", chunk_size=1500, chunk_overlap=200)
    print("✅ HR RAG pipeline setup complete.")

    # 3b. Technical RAG Pipeline
    print("\n⚙️  Setting up Technical RAG pipeline...")
    tech_retriever = create_rag_retriever("./tech_docs.txt", chunk_size=1200, chunk_overlap=150)
    print("✅ Technical RAG pipeline setup complete.")

except Exception as e:
    print(f"❌ Error setting up RAG pipelines: {e}")
    exit()


# 4. Define Tools for the Agents
# ------------------------------
query_expansion_prompt = ChatPromptTemplate.from_template("Rewrite the user query into 3 alternative versions to improve vector search recall.\n\nOriginal Query:\n{query}\n\nExpanded Queries:")
query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()

@tool
def hr_rag_tool(query: str) -> str:
    """Searches the HR knowledge base for company policies."""
    print(f"\n🔎 Original HR query: '{query}'")
    expanded_queries_str = query_expansion_chain.invoke({"query": query})
    all_queries = [query] + expanded_queries_str.strip().split('\n')
    print(f"🔍 Performing retrieval with expanded HR queries: {all_queries}")
    all_retrieved_docs = []
    for q in all_queries:
        all_retrieved_docs.extend(hr_retriever.invoke(q))
    unique_parent_contents = {doc.metadata['parent_content'] for doc in all_retrieved_docs}
    return f"Retrieved context:\n" + "\n\n".join(unique_parent_contents)

# --- NEW: Tech RAG Tool ---
@tool
def tech_rag_tool(query: str) -> str:
    """
    Searches the internal technical knowledge base. Use this for questions about
    company-specific projects, repositories, deployment, and coding standards.
    """
    print(f"\n🔎 Retrieving internal tech context for: '{query}'...")
    docs = tech_retriever.invoke(query)
    context = "\n\n".join(doc.metadata['parent_content'] for doc in docs)
    return f"Retrieved context:\n{context}"

@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary from Wikipedia for general knowledge technical questions."""
    print(f"\n🔎 Searching Wikipedia for: '{query}'...")
    try:
        return wikipedia.summary(query, auto_suggest=False, sentences=5)
    except Exception as e:
        return f"An error occurred fetching from Wikipedia: {e}"


# 5. Define All Agents
# ----------------------------------------
class GuardrailOutput(BaseModel):
    is_valid: bool; reasoning: str
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "Is the user's query about HR or Technical topics? Respond only with the structured output."), ("human", "Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

# HR Agent (no changes)
hr_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are an HR assistant. Use the `hr_rag_tool` to answer questions based ONLY on the retrieved context."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

# --- UPDATED: Technical Agent ---
# Now has two tools and must choose between them.
tech_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a specialist technical assistant. You have two tools:
    1. `tech_rag_tool`: For questions about internal company systems, code, projects like 'Phoenix', repositories, and deployment.
    2. `wiki_tool`: For general public technical knowledge, programming concepts, and open-source technologies.

    You must decide which tool is appropriate. If the question is about company specifics, use `tech_rag_tool`. Otherwise, use `wiki_tool`."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
tech_tools = [tech_rag_tool, wiki_tool] # Provide both tools
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)


class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "Is this query for 'HR' or 'Technical'? Respond only with the structured output."), ("human", "Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 6. Orchestrate the System (No changes here)
# ---------------------------------------------
async def run_agent_system(query: str):
    print("="*80)
    print(f"👤 User Query: {query}")
    print("="*80)
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed.")
    print("\n🚦 Running Triage Agent...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*80)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*80 + "\n")


# 7. Run Example Queries
# ----------------------
async def main():
    # Test 1: An internal tech question that should use `tech_rag_tool`
    await run_agent_system("Where is the repository for the Phoenix project's frontend?")

    # Test 2: A general tech question that should use `wiki_tool`
    await run_agent_system("What is FastAPI?")

    # Test 3: An HR question that should use `hr_rag_tool`
    await run_agent_system("What's the policy on compassionate leave?")


if __name__ == "__main__":
    if llm and hr_retriever and tech_retriever:
        try:
             await main()
        except NameError:
             asyncio.run(main())

✅ Gemini API Key configured successfully.

⚙️  Setting up HR RAG pipeline...
✅ HR RAG pipeline setup complete.

⚙️  Setting up Technical RAG pipeline...
✅ Technical RAG pipeline setup complete.
👤 User Query: Where is the repository for the Phoenix project's frontend?
🛡️  Running Guardrail Check...
❌ Guardrail Blocked Input. Reason: Unable to access external websites or specific file systems to check for the Phoenix project's frontend repository.

👤 User Query: What is FastAPI?
🛡️  Running Guardrail Check...
❌ Guardrail Blocked Input. Reason: The available tools lack the information to answer this question.

👤 User Query: What's the policy on compassionate leave?
🛡️  Running Guardrail Check...
✅ Guardrail Passed.

🚦 Running Triage Agent...
🎯 Specialist selected: HR

▶️  Invoking HR Agent...

--------------------------------------------------------------------------------
🤖 Final Answer:
I'm sorry, I cannot answer this question. The available tools lack the required
information.
--------

## HR pipeline with Post-Retrieval Re-ranking

In [5]:
!pip install flashrank -q
!pip install --upgrade langchain langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00


In [3]:
# 1. Install necessary libraries
# ---------------------------------
# !pip install langchain langchain-google-genai pydantic
# !pip install langchain-community faiss-cpu
# !pip install flashrank # Add this for the re-ranker

import os
import asyncio
import textwrap
from typing import List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank

# --- SETUP: HR POLICY FILE ---
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration.
- Up to 1 year of service: 2 weeks full pay, 2 weeks half pay.
- 1-3 years of service: 4 weeks full pay, 4 weeks half pay.
- Over 3 years of service: 8 weeks full pay, 8 weeks half pay.
A doctor's note is required for absences longer than 3 consecutive days.

## Compassionate Leave
Special leave for bereavement is available. Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member (spouse, child, parent, sibling).

## Public Holidays
The company observes all official public holidays in the country of employment. These days are paid leave and do not count against an employee's annual leave balance.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week with manager approval. WFH arrangements are based on job role and performance. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. Setup RAG Pipeline
# ---------------------
retriever = None
try:
    print("\n⚙️  Setting up HR RAG pipeline...")
    loader = TextLoader("./hr_policy.txt")
    docs = loader.load()
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    parent_documents = parent_splitter.split_documents(docs)
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Retrieve more docs initially for re-ranking
    print("✅ RAG pipeline setup complete.")
except Exception as e:
    print(f"❌ Error setting up RAG pipeline: {e}")
    exit()


# 4. Instantiate the Re-ranker
# ----------------------------
print("\n✨ Initializing Post-Retrieval Re-ranker...")
reranker = FlashrankRerank()
print("✅ Re-ranker initialized.")


# 5. Define the HR Tool with Re-ranking
# ---------------------------------------
@tool
def hr_rag_tool(query: str) -> str:
    """
    Searches the HR knowledge base for company policies to answer HR-related questions.
    """
    print(f"\n🔎 Retrieving initial documents for: '{query}'...")
    # 1. Retrieve initial set of documents. We retrieve more (k=5) to give the re-ranker more to work with.
    child_docs = retriever.invoke(query)

    # 2. NEW STEP: Re-rank the retrieved documents for relevance.
    print(f"✨ Re-ranking {len(child_docs)} retrieved documents...")
    reranked_docs = reranker.compress_documents(documents=child_docs, query=query)
    print(f"✅ Top {len(reranked_docs)} documents after re-ranking:")

    # 3. Extract parent content from the top-ranked documents
    unique_parent_contents = {doc.metadata['parent_content'] for doc in reranked_docs}

    # 4. Format the final context
    context = "\n\n".join(unique_parent_contents)
    return f"Retrieved and re-ranked context:\n{context}"


# 6. Define the HR Agent
# ----------------------
hr_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful H.R. assistant. You must use the `hr_rag_tool` to find information in the company policy document. Answer the user's question based ONLY on the final context provided by the tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)


# 7. Run the System
# -----------------
async def run_hr_system(query: str):
    print("="*80)
    print(f"👤 User Query: {query}")
    print("="*80)
    result = await hr_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*80)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*80 + "\n")


async def main():
    # A query where re-ranking can help distinguish between different types of leave
    await run_hr_system("I was sick for a week and my manager says I should have submitted a request in advance. Is that right?")
    # A query about getting money for WFH setup
    await run_hr_system("Do I get any money to set up my home office for remote work?")


if __name__ == "__main__":
    if llm and retriever:
        try:
            asyncio.run(main())
        except RuntimeError:
            # This handles running in a notebook where an event loop is already running.
            await main()

✅ Gemini API Key configured successfully.

⚙️  Setting up HR RAG pipeline...
✅ RAG pipeline setup complete.

✨ Initializing Post-Retrieval Re-ranker...


ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:00<00:00, 145MiB/s]


✅ Re-ranker initialized.
👤 User Query: I was sick for a week and my manager says I should have submitted a request in advance. Is that right?

--------------------------------------------------------------------------------
🤖 Final Answer:
I am sorry, I cannot answer this question. I do not have access to the company's
policy on this matter.  I need more information to assist you.
--------------------------------------------------------------------------------

👤 User Query: Do I get any money to set up my home office for remote work?

🔎 Retrieving initial documents for: 'Home office stipend policy'...
✨ Re-ranking 4 retrieved documents...
✅ Top 3 documents after re-ranking:

--------------------------------------------------------------------------------
🤖 Final Answer:
Based on the company policy, yes, there is a stipend provided for home office
setup.  However, the exact amount isn't specified here.  You should contact HR
for details on the stipend amount.
--------------------------

/tmp/ipython-input-20271066.py:163: RuntimeWarning: coroutine 'main' was never awaited
  await main()


## Post-Retrieval Re-ranking

In [5]:
# 1. Install necessary libraries
# ---------------------------------
# !pip install langchain langchain-google-genai wikipedia pydantic
# !pip install langchain-community faiss-cpu
# !pip install flashrank

import os
import asyncio
import textwrap
from typing import Literal, List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank

# --- SETUP: Create Dummy Knowledge Base Files ---

# HR Policy Document
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration. A doctor's note is required for absences longer than 3 consecutive days. After 1 year of service, employees get 4 weeks full pay.

## Compassionate Leave
Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)

# Technical Knowledge Base Document
tech_docs_content = """
# Internal Technical Documentation

## Project Phoenix: Frontend
- Repository: git.corp.example.com/phoenix/frontend-app
- Language: TypeScript, Framework: React
- Description: This is the main customer-facing web application.

## Project Phoenix: Backend
- Repository: git.corp.example.com/phoenix/backend-services
- Language: Python, Framework: FastAPI
- Description: These are the microservices that power the Phoenix frontend.

## Deployment Procedures
- Deployments to production must go through the CI/CD pipeline in Jenkins.
- Request a production deployment by creating a JIRA ticket with the 'DEVOPS' component.
"""
with open("tech_docs.txt", "w") as f:
    f.write(tech_docs_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. Setup RAG Pipelines
# ----------------------
hr_retriever = None
tech_retriever = None

# Helper function to create a RAG retriever
def create_rag_retriever(file_path: str) -> FAISS.as_retriever:
    loader = TextLoader(file_path)
    docs = loader.load()
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    parent_documents = parent_splitter.split_documents(docs)
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)
    # Retrieve more documents initially to feed into the re-ranker
    return vectorstore.as_retriever(search_kwargs={"k": 5})

try:
    print("\n⚙️  Setting up HR RAG pipeline...")
    hr_retriever = create_rag_retriever("./hr_policy.txt")
    print("✅ HR RAG pipeline setup complete.")

    print("\n⚙️  Setting up Technical RAG pipeline...")
    tech_retriever = create_rag_retriever("./tech_docs.txt")
    print("✅ Technical RAG pipeline setup complete.")

except Exception as e:
    print(f"❌ Error setting up RAG pipelines: {e}")
    exit()


# 4. Instantiate the Re-ranker
# ----------------------------
print("\n✨ Initializing Post-Retrieval Re-ranker...")
reranker = FlashrankRerank()
print("✅ Re-ranker initialized.")


# 5. Define All Tools
# -------------------
@tool
def hr_rag_tool(query: str) -> str:
    """Searches the HR knowledge base for company policies. Use for HR-related questions."""
    print(f"\n🔎 Retrieving initial HR documents for: '{query}'...")
    child_docs = hr_retriever.invoke(query)
    print(f"✨ Re-ranking {len(child_docs)} retrieved HR documents...")
    reranked_docs = reranker.compress_documents(documents=child_docs, query=query)
    unique_parent_contents = {doc.metadata['parent_content'] for doc in reranked_docs}
    return "Retrieved and re-ranked context:\n" + "\n\n".join(unique_parent_contents)

@tool
def tech_rag_tool(query: str) -> str:
    """Searches internal tech docs for company projects, repos, and standards."""
    print(f"\n🔎 Retrieving internal tech context for: '{query}'...")
    docs = tech_retriever.invoke(query)
    context = "\n\n".join(doc.metadata['parent_content'] for doc in docs)
    return f"Retrieved context:\n{context}"

@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary from Wikipedia for general knowledge technical questions."""
    print(f"\n🔎 Searching Wikipedia for: '{query}'...")
    try:
        return wikipedia.summary(query, auto_suggest=False, sentences=5)
    except Exception as e:
        return f"An error occurred fetching from Wikipedia: {e}"


# 6. Define All Agents
# --------------------

# Guardrail Agent
class GuardrailOutput(BaseModel):
    is_valid: bool; reasoning: str
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "Is the user's query about HR or Technical topics? Respond only with the structured output."), ("human", "Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

# HR Agent (with advanced RAG tool)
hr_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are an HR assistant. Use the `hr_rag_tool` to answer questions based ONLY on the retrieved context."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

# Technical Agent (with two tools)
tech_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technical assistant with two tools: `tech_rag_tool` for internal company tech info, and `wiki_tool` for public general tech knowledge. You must choose the appropriate tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
tech_tools = [tech_rag_tool, wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)

# Triage Agent
class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "Is this query for 'HR' or 'Technical'? Respond only with the structured output."), ("human", "Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 7. Orchestrate the System
# -------------------------
async def run_agent_system(query: str):
    print("="*80)
    print(f"👤 User Query: {query}")
    print("="*80)
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed.")
    print("\n🚦 Running Triage Agent...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*80)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*80 + "\n")


# 8. Run Example Queries
# ----------------------
async def main():
    # Test 1: An internal tech question
    await run_agent_system("Where can I find the backend repo for Project Phoenix?")
    # Test 2: A general tech question
    await run_agent_system("What is the difference between TypeScript and JavaScript?")
    # Test 3: An HR question
    await run_agent_system("If I'm sick for more than 3 days, what do I need to do?")
    # Test 4: An out-of-scope question
    await run_agent_system("What are the best tourist spots in Mumbai?")


if __name__ == "__main__":
    if llm and hr_retriever and tech_retriever:
        try:
            asyncio.run(main())
        except RuntimeError:
            await main()

✅ Gemini API Key configured successfully.

⚙️  Setting up HR RAG pipeline...
✅ HR RAG pipeline setup complete.

⚙️  Setting up Technical RAG pipeline...
✅ Technical RAG pipeline setup complete.

✨ Initializing Post-Retrieval Re-ranker...
✅ Re-ranker initialized.
👤 User Query: Where can I find the backend repo for Project Phoenix?
🛡️  Running Guardrail Check...
❌ Guardrail Blocked Input. Reason: Query is about Technical topics

👤 User Query: What is the difference between TypeScript and JavaScript?
🛡️  Running Guardrail Check...
✅ Guardrail Passed.

🚦 Running Triage Agent...
🎯 Specialist selected: Technical

▶️  Invoking Technical Agent...

🔎 Searching Wikipedia for: 'What is the difference between TypeScript and JavaScript?'...

--------------------------------------------------------------------------------
🤖 Final Answer:
I cannot answer this question using the available tools.  The `wiki_tool` failed
to find relevant information.  To get an answer, I would need access to a tool
that

/tmp/ipython-input-4172282171.py:237: RuntimeWarning: coroutine 'main' was never awaited
  await main()


# Final

In [6]:
# 1. Install all necessary libraries
# ------------------------------------
# !pip install langchain langchain-google-genai wikipedia pydantic
# !pip install langchain-community faiss-cpu
# !pip install flashrank

import os
import asyncio
import textwrap
from typing import Literal, List

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
import wikipedia

# --- RAG Specific Imports ---
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank

# --- SETUP: Create Dummy Knowledge Base Files ---

# HR Policy Document
hr_policy_content = """
# Company Leave Policy

## Annual Leave
Full-time staff are entitled to 24 days of annual leave per year. This accrues at a rate of 2 days per month. Up to 5 unused days may be carried forward into the next year with manager approval. Requests for leave must be submitted through the HR portal at least two weeks in advance.

## Sick Leave
Sick leave is for personal illness or injury. It is determined by service duration. A doctor's note is required for absences longer than 3 consecutive days. After 1 year of service, employees get 4 weeks full pay.

## Compassionate Leave
Employees can take up to 5 days of paid compassionate leave per year for the loss of a close family member.

## Work From Home (WFH) Policy
Employees may work from home up to 2 days per week. The company provides a stipend for home office setup. All company IT and security policies must be adhered to while working remotely.
"""
with open("hr_policy.txt", "w") as f:
    f.write(hr_policy_content)

# Technical Knowledge Base Document
tech_docs_content = """
# Internal Technical Documentation

## Project Phoenix: Frontend
- Repository: git.corp.example.com/phoenix/frontend-app
- Language: TypeScript, Framework: React
- Description: This is the main customer-facing web application.

## Project Phoenix: Backend
- Repository: git.corp.example.com/phoenix/backend-services
- Language: Python, Framework: FastAPI
- Description: These are the microservices that power the Phoenix frontend.

## Deployment Procedures
- Deployments to production must go through the CI/CD pipeline in Jenkins.
- Request a production deployment by creating a JIRA ticket with the 'DEVOPS' component.
"""
with open("tech_docs.txt", "w") as f:
    f.write(tech_docs_content)
# ------------------------------------


# 2. Configure API Key
# --------------------
llm = None
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    llm.invoke("Test query")
    print("✅ Gemini API Key configured successfully.")
except Exception as e:
    print(f"❌ Error configuring Gemini API: {e}")
    exit()


# 3. Setup RAG Pipelines
# ----------------------
hr_retriever = None
tech_retriever = None

# Helper function to create a RAG retriever
def create_rag_retriever(file_path: str) -> FAISS.as_retriever:
    loader = TextLoader(file_path)
    docs = loader.load()
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    parent_documents = parent_splitter.split_documents(docs)
    child_documents = []
    for parent_doc in parent_documents:
        child_docs = child_splitter.split_text(parent_doc.page_content)
        for child_doc_text in child_docs:
            child_documents.append(
                Document(page_content=child_doc_text, metadata={"parent_content": parent_doc.page_content})
            )
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(child_documents, embeddings)
    # Retrieve more documents initially (k=5) to feed into the re-ranker
    return vectorstore.as_retriever(search_kwargs={"k": 5})

try:
    print("\n⚙️  Setting up HR RAG pipeline...")
    hr_retriever = create_rag_retriever("./hr_policy.txt")
    print("✅ HR RAG pipeline setup complete.")

    print("\n⚙️  Setting up Technical RAG pipeline...")
    tech_retriever = create_rag_retriever("./tech_docs.txt")
    print("✅ Technical RAG pipeline setup complete.")

except Exception as e:
    print(f"❌ Error setting up RAG pipelines: {e}")
    exit()


# 4. Instantiate Re-ranker and Query Expansion Chain
# --------------------------------------------------
print("\n✨ Initializing Re-ranker and Query Expansion chain...")
reranker = FlashrankRerank()
query_expansion_prompt = ChatPromptTemplate.from_template("Rewrite the user query into 3 alternative versions to improve vector search recall.\n\nOriginal Query:\n{query}\n\nExpanded Queries:")
query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()
print("✅ Re-ranker and Query Expansion chain initialized.")


# 5. Define All Tools with Full RAG Capabilities
# ----------------------------------------------
@tool
def hr_rag_tool(query: str) -> str:
    """Searches the HR knowledge base for company policies. Use for HR-related questions."""
    print(f"\n[HR Tool] ➡️ Query: '{query}'")
    # 1. Pre-retrieval: Query Expansion
    expanded_queries_str = query_expansion_chain.invoke({"query": query})
    all_queries = [query] + expanded_queries_str.strip().split('\n')
    print(f"[HR Tool] 🔍 Expanded queries: {all_queries}")
    # 2. Retrieval
    all_retrieved_docs = []
    for q in all_queries:
        all_retrieved_docs.extend(hr_retriever.invoke(q))
    # 3. Post-retrieval: Re-ranking
    print(f"[HR Tool] ✨ Re-ranking {len(all_retrieved_docs)} documents...")
    reranked_docs = reranker.compress_documents(documents=all_retrieved_docs, query=query)
    unique_parent_contents = {doc.metadata['parent_content'] for doc in reranked_docs}
    return "Retrieved and re-ranked context:\n" + "\n\n".join(unique_parent_contents)

@tool
def tech_rag_tool(query: str) -> str:
    """Searches internal tech docs for company projects, repos, and standards."""
    print(f"\n[Tech Tool] ➡️ Query: '{query}'")
    # 1. Pre-retrieval: Query Expansion
    expanded_queries_str = query_expansion_chain.invoke({"query": query})
    all_queries = [query] + expanded_queries_str.strip().split('\n')
    print(f"[Tech Tool] 🔍 Expanded queries: {all_queries}")
    # 2. Retrieval
    all_retrieved_docs = []
    for q in all_queries:
        all_retrieved_docs.extend(tech_retriever.invoke(q))
    # 3. Post-retrieval: Re-ranking
    print(f"[Tech Tool] ✨ Re-ranking {len(all_retrieved_docs)} documents...")
    reranked_docs = reranker.compress_documents(documents=all_retrieved_docs, query=query)
    unique_parent_contents = {doc.metadata['parent_content'] for doc in reranked_docs}
    return f"Retrieved and re-ranked context:\n" + "\n\n".join(unique_parent_contents)

@tool
def wiki_tool(query: str) -> str:
    """Fetches a summary from Wikipedia for general knowledge technical questions."""
    print(f"\n[Wiki Tool] ➡️ Query: '{query}'...")
    try:
        return wikipedia.summary(query, auto_suggest=False, sentences=5)
    except Exception as e:
        return f"An error occurred fetching from Wikipedia: {e}"


# 6. Define All Agents
# --------------------
# Guardrail Agent
class GuardrailOutput(BaseModel):
    is_valid: bool; reasoning: str
guardrail_prompt = ChatPromptTemplate.from_messages([("system", "Is the user's query about HR or Technical topics? Respond only with the structured output."), ("human", "Query: {query}")])
guardrail_agent_runnable = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

# HR Agent
hr_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are an HR assistant. Use the `hr_rag_tool` to answer questions based ONLY on the retrieved context."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
hr_tools = [hr_rag_tool]
hr_agent = create_tool_calling_agent(llm, hr_tools, hr_agent_prompt)
hr_agent_executor = AgentExecutor(agent=hr_agent, tools=hr_tools, verbose=False)

# Technical Agent
tech_agent_prompt = ChatPromptTemplate.from_messages([("system", "You are a technical assistant with two tools: `tech_rag_tool` for internal company tech info, and `wiki_tool` for public general tech knowledge. You must choose the appropriate tool."), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
tech_tools = [tech_rag_tool, wiki_tool]
tech_agent = create_tool_calling_agent(llm, tech_tools, tech_agent_prompt)
tech_agent_executor = AgentExecutor(agent=tech_agent, tools=tech_tools, verbose=False)

# Triage Agent
class TriageDecision(BaseModel):
    agent: Literal["HR", "Technical"]
triage_prompt = ChatPromptTemplate.from_messages([("system", "Is this query for 'HR' or 'Technical'? Respond only with the structured output."), ("human", "Query: {query}")])
triage_agent_runnable = triage_prompt | llm.with_structured_output(TriageDecision)


# 7. Orchestrate the System
# -------------------------
async def run_agent_system(query: str):
    print("="*80)
    print(f"👤 User Query: {query}")
    print("="*80)
    print("🛡️  Running Guardrail Check...")
    guardrail_result = await guardrail_agent_runnable.ainvoke({"query": query})
    if not guardrail_result.is_valid:
        print(f"❌ Guardrail Blocked Input. Reason: {guardrail_result.reasoning}\n")
        return
    print(f"✅ Guardrail Passed.")
    print("\n🚦 Running Triage Agent...")
    triage_decision = await triage_agent_runnable.ainvoke({"query": query})
    selected_agent = triage_decision.agent
    print(f"🎯 Specialist selected: {selected_agent}")
    print(f"\n▶️  Invoking {selected_agent} Agent...")
    if selected_agent == "HR":
        result = await hr_agent_executor.ainvoke({"input": query})
    else:
        result = await tech_agent_executor.ainvoke({"input": query})
    final_answer = result.get('output', 'Sorry, I could not process your request.')
    print("\n" + "-"*80)
    print("🤖 Final Answer:")
    print(textwrap.fill(final_answer, width=80))
    print("-"*80 + "\n")


# 8. Run Example Queries
# ----------------------
async def main():
    # Test 1: Internal tech question -> Triage to Tech -> Tech Agent uses tech_rag_tool
    await run_agent_system("Where can I find the backend repo for Project Phoenix?")

    # Test 2: General tech question -> Triage to Tech -> Tech Agent uses wiki_tool
    await run_agent_system("What is React?")

    # Test 3: HR question -> Triage to HR -> HR Agent uses hr_rag_tool
    await run_agent_system("Do I get paid for compassionate leave?")

    # Test 4: Out-of-scope question -> Blocked by Guardrail
    await run_agent_system("What's the weather like today?")


if __name__ == "__main__":
    if llm and hr_retriever and tech_retriever:
        try:
            asyncio.run(main())
        except RuntimeError:
            await main()

✅ Gemini API Key configured successfully.

⚙️  Setting up HR RAG pipeline...
✅ HR RAG pipeline setup complete.

⚙️  Setting up Technical RAG pipeline...
✅ Technical RAG pipeline setup complete.

✨ Initializing Re-ranker and Query Expansion chain...
✅ Re-ranker and Query Expansion chain initialized.
👤 User Query: Where can I find the backend repo for Project Phoenix?
🛡️  Running Guardrail Check...
❌ Guardrail Blocked Input. Reason: Query is about Technical topics

👤 User Query: What is React?
🛡️  Running Guardrail Check...
❌ Guardrail Blocked Input. Reason: The query is about a technical topic

👤 User Query: Do I get paid for compassionate leave?
🛡️  Running Guardrail Check...
✅ Guardrail Passed.

🚦 Running Triage Agent...
🎯 Specialist selected: HR

▶️  Invoking HR Agent...

--------------------------------------------------------------------------------
🤖 Final Answer:
I'm sorry, I cannot answer this question. The available tools lack the
information to address your query.
------------

/tmp/ipython-input-1362040401.py:256: RuntimeWarning: coroutine 'main' was never awaited
  await main()
